## Imports

In [3]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Links

Miami-Dade data download: https://gis-mdc.opendata.arcgis.com/datasets/MDC::property-boundary-view/about
DOR Use code descriptions: https://gis-mdc.opendata.arcgis.com/datasets/MDC::dor-code/explore

Palm Beach data download: https://opendata2-pbcgov.opendata.arcgis.com/datasets/PBCGOV::property-information-table/explore

Broward Property Appraiser "Contact Us" page: https://bcpa.net/phone.asp

## Data read-in

In [4]:
MiamiDade = gpd.read_file('Property_Boundary_View.geojson')
# PalmBeach = pd.read_csv('Property_Information_Table.csv',dtype='str')

In [ ]:
# Broward = gpd.read_file('broward_2022pin/broward_2022pin.shp')

## Data Check

In [5]:
MD_agri = MiamiDade[MiamiDade['DOR_DESC'].str.contains('AGRI',na=False)]

In [ ]:
# PB_agri = PalmBeach[PalmBeach['PROPERTY_USE'].str.contains('AGRI',na=False)]

## Make Map

In [6]:
MD_agri['COLOR'] = 'blue'
# PB_agri['COLOR'] = 'orange'

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
def popup_html_MD(row):
    address = row['TRUE_SITE_ADDR']
    folio = row['FOLIO']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Folio: </strong>{}'''.format(folio) + '''<br>
    </html>
    '''
    return html

# def popup_html_PB(row):
#     address = row['SITE_ADDR_STR']
#     folio = row['PARCEL_NUMBER']
    
#     html = '''<!DOCTYPE html>
#     <html>
#     <strong>Address: </strong>{}'''.format(address) + '''<br>
#     <strong>Folio: </strong>{}'''.format(folio) + '''<br>
#     </html>
#     '''
#     return html

# def popup_html_BC(row):
#     folio = row['PARCELNO']
    
#     html = '''<!DOCTYPE html>
#     <html>
#     <strong>Folio: </strong>{}'''.format(folio) + '''<br>
#     </html>
#     '''
#     return html

## Geocode

In [ ]:
# %store -r google_maps_API_Key
# gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [ ]:
# PB_agri['full_address'] = PB_agri['SITE_ADDR_STR'] + ' Palm Beach County, FL'

In [ ]:
# def geocode(add):
#     g = gmaps_key.geocode(add)
#     lat = g[0]["geometry"]["location"]["lat"]
#     lng = g[0]["geometry"]["location"]["lng"]
#     return (lat, lng)

# PB_agri['geocoded'] = PB_agri['full_address'].apply(geocode)

In [8]:
MD_agri[['X_COORD','Y_COORD']] = MD_agri[['X_COORD','Y_COORD']].astype(float)
MD_agri['X_COORD'] = MD_agri['X_COORD'] * -1

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
MD_agri.to_file('MD_agri.geojson', driver='GeoJSON')

In [ ]:
### Create map container ###
m = folium.Map(location=[MD_agri.centroid.y.mean(), MD_agri.centroid.x.mean()], zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'South Florida Agricultural Properties')

# Add the GeoJSON layer to the map
folium.GeoJson(MD_agri.to_json(),
               name='geojson',
               popup=folium.features.GeoJsonPopup(fields=['TRUE_SITE_ADDR', 'FOLIO'])
              ).add_to(m)

# folium.GeoJson(Broward.to_json(),
#                name='geojson',
#                popup=folium.features.GeoJsonPopup(fields=['PARCELNO'])
#               ).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

# Function to handle search and zoom to the entered address
def zoom_to_address(address):
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(address)
    if location:
        # Zoom to the location
        m.fit_bounds([[location.latitude, location.longitude], [location.latitude, location.longitude]])
        # Add a marker at the location
        folium.Marker(
            [location.latitude, location.longitude],
            popup=address
        ).add_to(m)

# Create a Tkinter window
root = tk.Tk()

# Prompt the user to enter an address
address = simpledialog.askstring("Address", "Enter an address")

# Zoom to the entered address
zoom_to_address(address)
            
# Display map
m

In [ ]:
m.save('index.html')

## Map URL Snagger

In [11]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [12]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_agricultural_maps
